### Test loading and manipulaitng Argoverse Lidar datasets

In [ ]:
import av2 #ARGOVERSE API ONLY WORKS WITH PYTHON 3.8+
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import os
import sys
import inspect
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
from matplotlib import pyplot as plt

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
from ICET_spherical import ICET

%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 180


In [ ]:
#old
# fn1 = "D:/sensor/train/00a6ffc1-6ce9-3bc3-a060-6006e9893a1a/sensors/lidar/315967376859506000.feather"
# fn2 = "D:/sensor/train/00a6ffc1-6ce9-3bc3-a060-6006e9893a1a/sensors/lidar/315967376959702000.feather"

#new urban canyon
fn1 = "D:/sensor/train/087fec73-1a0c-399a-9292-cc2cf99dc97f/sensors/lidar/315975422260069000.feather"
fn2 = "D:/sensor/train/087fec73-1a0c-399a-9292-cc2cf99dc97f/sensors/lidar/315975422359602000.feather"

    
df1 = feather.read_feather(fn1)
df2 = feather.read_feather(fn2)
df1

In [ ]:
#get ground truth for trajectory file
# gtfn = "D:/sensor/train/00a6ffc1-6ce9-3bc3-a060-6006e9893a1a/city_SE3_egovehicle.feather" #old
# gtfn = "D:/sensor/train/087fec73-1a0c-399a-9292-cc2cf99dc97f/city_SE3_egovehicle.feather" #new canyon
gtfn = "D:/sensor/train/04973bcf-fc64-367c-9642-6d6c5f363b61/city_SE3_egovehicle.feather" #desert suburb

gt = feather.read_feather(gtfn)#.to_numpy()
# print(gt)
# print(np.shape(gt))
time = gt[{"timestamp_ns"}].to_numpy()
x = gt[{"tx_m"}].to_numpy()
y = gt[{"ty_m"}].to_numpy()
z = gt[{"tz_m"}].to_numpy()
# gt = np.array([x, y, z])[:,:,0].T #was this (just xyz)

#Need angles too! ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
qw = gt[{"qw"}].to_numpy()
qx = gt[{"qx"}].to_numpy()
qy = gt[{"qy"}].to_numpy()
qz = gt[{"qz"}].to_numpy()
#convert quaternions to euler angles
from scipy.spatial.transform import Rotation
quats = np.array([qw, qx, qy, qz])[:,:,0].T
# print(quats)
rot = Rotation.from_quat(quats)
rot_euler = rot.as_euler('xyz', degrees=True)
# print(rot_euler)
# print(np.diff(rot_euler, axis = 0))

gt = np.concatenate([x, y, z, rot_euler], axis = 1) #test
# print(np.diff(x, axis = 0))
# print(np.diff(gt, axis = 0)[0])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# np.sum(np.diff(gt, axis = 0)[:10], axis = 0)

In [ ]:
#plot all points in gt
fig, ax = plt.subplots()
ax.scatter(gt[:,0], gt[:,1])
ax.scatter(gt_lidar[:,0], gt_lidar[:,1], marker = 'x')

In [ ]:
#loop though all ~150 frame lidar datasets
path = "D:/sensor/train/"
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
# loop through each folder in subfolder paths
# for i in range(len(list_subfolders_with_paths)):
#     print(list_subfolders_with_paths[i])

#list all files in path
# path = "D:/sensor/train/087fec73-1a0c-399a-9292-cc2cf99dc97f/sensors/lidar/" #urabn canyon
path = "D:/sensor/train/04973bcf-fc64-367c-9642-6d6c5f363b61/sensors/lidar/" #desert suburb
files = os.listdir(path)
# files[0]
# # files[1:]
# print(files[0])
# for i in files[1:]:
#     print(i)

In [ ]:
#loop though each lidar frame 
# path = "D:/sensor/train/00a6ffc1-6ce9-3bc3-a060-6006e9893a1a/sensors/lidar/" #big city
# path = "D:/sensor/train/087fec73-1a0c-399a-9292-cc2cf99dc97f/sensors/lidar/" #urabn canyon
path = "D:/sensor/train/04973bcf-fc64-367c-9642-6d6c5f363b61/sensors/lidar/" #desert suburb

# gt_lidar = np.zeros([0,3])
gt_lidar = np.zeros([0,6])

count = 0
for file in os.listdir(path):
    #find ground truth pose recorded closest to this lidar timestamp
    t_lidar = int(file[:-8])
#     print(t_lidar)
#     print(np.where(time == t_lidar))
#     print(gt[18])
#     print(gt[np.where(time == t_lidar)[0]])
    gt_lidar = np.append(gt_lidar, gt[np.where(time == t_lidar)[0]], axis = 0)
#     print(gt_lidar)

    #read in point cloud
#     f = feather.read_feather(path + file)
    count += 1
#     if count == 10:
#         break
        
# print(gt_lidar)

dgt_lidar = np.diff(gt_lidar, axis = 0)
# print(dgt_lidar)
# print(np.sqrt(np.sum(dgt_lidar**2, axis = 1))) #absolute movement per frame in horizontal plane

np.savetxt("spherical_paper/Argoverse_results/suburb/gt_lidar.txt", gt_lidar)
print(gt_lidar)

In [ ]:
pts1 = df1[['x', 'y', 'z']].to_numpy()
pts2 = df2[['x', 'y', 'z']].to_numpy()
print(pts1) #need to swqp x and z for some reason
# pts1 = np.array([pts1[:,2], pts1[:,0], pts1[:,1]]).T
# pts2 = np.array([pts2[:,2], pts2[:,0], pts2[:,1]]).T
# print("\n", pts1) #need to swqp x and z for some reason


In [ ]:
ground = 0.3 
# ground = -10 #debug
pts1 = pts1[pts1[:,2] > ground ]
pts2 = pts2[pts2[:,2] > ground ]

it1 = ICET(cloud1 = pts1, cloud2 = pts2,  fid = 70, draw = True, 
            niter = 12, group= 2, RM = False, DNN_filter = False)

it1.plt.show(it1.disp, "Figure 2", resetcam = False)
ViewInteractiveWidget(it1.plt.window)